In [1]:
# import essential libraries
import numpy as np
import pandas as pd
import gzip

## Export to CSV

### 10 Features - 1 Label

We'll only keep 

**10 Features:**
* age
* sex
* height
* weight
* heart_axis
* validated_by
* second_opinion
* validated_by_human
* pacemaker
* strat_fold

**1 Label:**
* ritmi

We'll specifically drop missing values for the height and weight columns. Then we'll fill the missing values for the age column with the mean values. For the other columns, we'll fill in the missing values with either 0 or 'missing'. Therefore, we'll end up having 1803 data points with 10 features and 1 label, then export to csv file.

In [2]:
# read in csv
df = pd.read_csv('coorteeqsrafva.csv', sep=';', header=0, index_col=0)
df.head()

,diagnosi,ecg_id,ritmi,patient_id,age,sex,height,weight,nurse,site,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,STACH,10900,VA,15654.0,54.0,0,NaN,NaN,0.0,0.0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,6,records100/10000/10900_lr,records500/10000/10900_hr
1,AFLT,10900,AF,15654.0,54.0,0,NaN,NaN,0.0,0.0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,6,records100/10000/10900_lr,records500/10000/10900_hr
2,SR,8209,SR,12281.0,55.0,0,NaN,NaN,1.0,2.0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,10,records100/08000/08209_lr,records500/08000/08209_hr
3,STACH,17620,VA,2007.0,29.0,1,164.0,56.0,7.0,1.0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,1,records100/17000/17620_lr,records500/17000/17620_hr
4,SBRAD,12967,VA,8685.0,57.0,0,NaN,NaN,0.0,0.0,...,False,NaN,", I-AVR,",NaN,NaN,NaN,NaN,1,records100/12000/12967_lr,records500/12000/12967_hr


In [3]:
# drop columns
sub_df = df.drop(columns=['diagnosi', 'ecg_id', 'patient_id', 'nurse', 'site', 'device', 'recording_date', 'report', 'scp_codes', 'infarction_stadium1', 'infarction_stadium2', 'initial_autogenerated_report', 'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems', 'extra_beats', 'filename_lr', 'filename_hr'])
sub_df.head()

,ritmi,age,sex,height,weight,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold
0,VA,54.0,0,NaN,NaN,MID,NaN,False,False,NaN,6
1,AF,54.0,0,NaN,NaN,MID,NaN,False,False,NaN,6
2,SR,55.0,0,NaN,NaN,LAD,1.0,False,True,NaN,10
3,VA,29.0,1,164.0,56.0,NaN,0.0,False,True,NaN,1
4,VA,57.0,0,NaN,NaN,MID,NaN,False,False,NaN,1


In [4]:
# get info for columns
sub_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6428 entries, 0 to 6427
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ritmi               6428 non-null   object 
 1   age                 6394 non-null   float64
 2   sex                 6428 non-null   int64  
 3   height              1866 non-null   float64
 4   weight              2428 non-null   float64
 5   heart_axis          4124 non-null   object 
 6   validated_by        3676 non-null   float64
 7   second_opinion      6428 non-null   bool   
 8   validated_by_human  6428 non-null   bool   
 9   pacemaker           294 non-null    object 
 10  strat_fold          6428 non-null   int64  
dtypes: bool(2), float64(4), int64(2), object(3)
memory usage: 514.7+ KB


In [5]:
# drop missing values for height and weight
sub_df = sub_df[sub_df['height'].notna()]
sub_df = sub_df[sub_df['weight'].notna()]

# fill missing values for age with the mean
sub_df['age'].fillna((sub_df['age'].mean()), inplace=True)

# fill missing values for validated_by with 0
sub_df['validated_by'] = sub_df['validated_by'].fillna(0)

# fill missing values for heart_axis with missing
sub_df['heart_axis'] = sub_df['heart_axis'].fillna('Missing')

# fill missing values for pacemaker with missing 
sub_df['pacemaker'] = sub_df['pacemaker'].fillna('Missing')

In [6]:
# get info for columns
sub_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1803 entries, 3 to 6426
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ritmi               1803 non-null   object 
 1   age                 1803 non-null   float64
 2   sex                 1803 non-null   int64  
 3   height              1803 non-null   float64
 4   weight              1803 non-null   float64
 5   heart_axis          1803 non-null   object 
 6   validated_by        1803 non-null   float64
 7   second_opinion      1803 non-null   bool   
 8   validated_by_human  1803 non-null   bool   
 9   pacemaker           1803 non-null   object 
 10  strat_fold          1803 non-null   int64  
dtypes: bool(2), float64(4), int64(2), object(3)
memory usage: 144.4+ KB


In [7]:
# map categorical values to numerical values
sub_df['ritmi'] = sub_df['ritmi'].map({'SR': 0, 'AF': 1, 'VA': 2}).values
sub_df['second_opinion'] = sub_df['second_opinion'].map({False: 0, True:1}).values
sub_df['validated_by_human'] = sub_df['validated_by_human'].map({False: 0, True:1}).values
sub_df['heart_axis'] = sub_df['heart_axis'].map({'Missing':0, 'LAD':1, 'AXL':2, 'MID':3, 'RAD':4, 'ALAD':5, 'AXR':6}).values
sub_df['pacemaker'] = sub_df['pacemaker'].map({'Missing':0, 'PACE????, nan': 0, '?, nan':0, 'ja, pacemaker':1}).values

# reset index
sub_df = sub_df.reset_index(drop=True)
sub_df

,ritmi,age,sex,height,weight,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold
0,2,29.0,1,164.0,56.0,0,0.0,0,1,0,1
1,0,59.0,0,156.0,75.0,0,0.0,0,1,0,9
2,2,84.0,1,152.0,51.0,0,0.0,0,1,0,7
3,0,79.0,0,172.0,66.0,0,0.0,0,1,0,5
4,1,67.0,0,178.0,73.0,4,0.0,0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...
1798,0,76.0,0,166.0,67.0,0,0.0,0,1,0,4
1799,2,78.0,1,176.0,76.0,1,4.0,0,1,0,1
1800,2,81.0,0,178.0,70.0,1,0.0,0,1,0,4
1801,2,88.0,0,152.0,45.0,0,0.0,0,1,0,10


In [8]:
# export to csv
sub_df.to_csv('training_11_features.csv', index=False)

### 13 Features - 1 Label

We'll only keep 

**13 Features:**
* age
* sex
* height
* weight
* nurse
* site
* device
* heart_axis
* validated_by
* second_opinion
* validated_by_human
* pacemaker
* strat_fold

**1 Label:** 
* ritmi

We'll specifically fill missing values with the mean values for the age, height, weight columns. In addition, we'll also fill missing values with 0 for the nurse, site, validated_by, heart_axis, and pacemaker columns. Therefore, we'll end up having 6366 data points with 13 features and 1 label, then export to csv file.

In [9]:
# read in csv file
df = pd.read_csv('coorteeqsrafva.csv', sep=';', header=0, index_col=0)
df.head()

,diagnosi,ecg_id,ritmi,patient_id,age,sex,height,weight,nurse,site,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,STACH,10900,VA,15654.0,54.0,0,NaN,NaN,0.0,0.0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,6,records100/10000/10900_lr,records500/10000/10900_hr
1,AFLT,10900,AF,15654.0,54.0,0,NaN,NaN,0.0,0.0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,6,records100/10000/10900_lr,records500/10000/10900_hr
2,SR,8209,SR,12281.0,55.0,0,NaN,NaN,1.0,2.0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,10,records100/08000/08209_lr,records500/08000/08209_hr
3,STACH,17620,VA,2007.0,29.0,1,164.0,56.0,7.0,1.0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,1,records100/17000/17620_lr,records500/17000/17620_hr
4,SBRAD,12967,VA,8685.0,57.0,0,NaN,NaN,0.0,0.0,...,False,NaN,", I-AVR,",NaN,NaN,NaN,NaN,1,records100/12000/12967_lr,records500/12000/12967_hr


In [10]:
# drop columns
sub_df = df.drop(columns=['diagnosi', 'ecg_id', 'patient_id', 'recording_date', 'report', 'scp_codes', 'infarction_stadium1', 'infarction_stadium2', 'initial_autogenerated_report', 'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems', 'extra_beats', 'filename_lr', 'filename_hr'])
sub_df.head()

,ritmi,age,sex,height,weight,nurse,site,device,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold
0,VA,54.0,0,NaN,NaN,0.0,0.0,CS100 3,MID,NaN,False,False,NaN,6
1,AF,54.0,0,NaN,NaN,0.0,0.0,CS100 3,MID,NaN,False,False,NaN,6
2,SR,55.0,0,NaN,NaN,1.0,2.0,CS-12,LAD,1.0,False,True,NaN,10
3,VA,29.0,1,164.0,56.0,7.0,1.0,AT-6 C 5.6,NaN,0.0,False,True,NaN,1
4,VA,57.0,0,NaN,NaN,0.0,0.0,CS100 3,MID,NaN,False,False,NaN,1


In [11]:
# fill missing values with mean values for the age, height, weight columns
sub_df['age'].fillna(value=sub_df['age'].mean(), inplace=True)
sub_df['height'].fillna(value=sub_df['height'].mean(), inplace=True)
sub_df['weight'].fillna(value=sub_df['weight'].mean(), inplace=True)

# fill missing values with 0 for these columns
sub_df['nurse'] = sub_df['nurse'].fillna(0)
sub_df['site'] = sub_df['site'].fillna(0)
sub_df['validated_by'] = sub_df['validated_by'].fillna(0)
sub_df['heart_axis'] = sub_df['heart_axis'].fillna('Missing')
sub_df['pacemaker'] = sub_df['pacemaker'].fillna('Missing')

In [12]:
# map categorical values to numerical values
sub_df['ritmi'] = sub_df['ritmi'].map({'SR': 0, 'AF': 1, 'VA': 2}).values
sub_df['second_opinion'] = sub_df['second_opinion'].map({False: 0, True:1}).values
sub_df['validated_by_human'] = sub_df['validated_by_human'].map({False: 0, True:1}).values
sub_df['heart_axis'] = sub_df['heart_axis'].map({'Missing':0, 'LAD':1, 'AXL':2, 'MID':3, 'RAD':4, 'ALAD':5, 'AXR':6}).values
sub_df['pacemaker'] = sub_df['pacemaker'].map({'Missing':0, 'PACE????, nan': 0, '?, nan':0, 'ja, pacemaker':1}).values
sub_df['device'] = sub_df['device'].map({'CS100    3':0, 'CS-12': 1, 'AT-6 C 5.5':2, 'CS-12   E':3, 'AT-6     6':4, 'AT-60    3':5, 'AT-6 C 5.8':6, 'AT-6 C':7, 'AT-6 C 5.0':8, 'AT-6 C 5.3':9, 'AT-6 C 5.6':10}).values

In [13]:
# check sub_df
print(sub_df.shape)
sub_df.head()

(6428, 14)


,ritmi,age,sex,height,weight,nurse,site,device,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold
0,2,54.0,0,166.796356,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
1,1,54.0,0,166.796356,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
2,0,55.0,0,166.796356,69.841845,1.0,2.0,1,1.0,1.0,0,1,0.0,10
3,2,29.0,1,164.000000,56.000000,7.0,1.0,10,0.0,0.0,0,1,0.0,1
4,2,57.0,0,166.796356,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,1


In [14]:
# export to csv
sub_df.to_csv('training_13_features.csv', index=False)

### Compress 3D Numpy Array Features

Since the numpy data file is too large, we'll try to reduce the size of it.

In [15]:
# compress numpy array file and export
f = gzip.GzipFile("compressed_npy.gz", "w")
ecg_arr = np.load('ecgeq-500hzsrfava.npy')
np.save(file=f, arr=ecg_arr)
f.close()

### 3D Numpy Arr to CSV

After reading in the compressed numpy data file, we will only get 6428 layers, 700 rows, and 12 columns to reduce the size of the numpy data. After that, we will transform the 3D array to 2D array and then convert it to a dataframe. The converted dataframe has 4361843 rows and 13 columns (12 leads and 1 index column). Afterwards, we will read in the csv file created above, which consists of 13 features. We use this dataframe to merge with the converted dataframe to get 13 more features. As a result, we have a total of 25 features and 1 label. The final dataframe will have 4319176 rows and 26 columns. 

**25 Features:**
* I
* II
* III
* aVF
* aVR
* aVL
* V1
* V2
* V3
* V4
* V5
* V6
* age
* sex
* height
* weight
* nurse
* site
* device
* heart_axis
* validated_by
* second_opinion
* validated_by_human
* pacemaker
* strat_fold

**1 Label:**
* ritmi

In [16]:
# read in the compressed numpy file
f = gzip.GzipFile('compressed_npy.gz', "r")
X = np.load(f)

# get 6428 layers, 700 rows and 12 columns
X = X[:,:700,:]
X.shape

(6428, 700, 12)

In [17]:
# convert 3d array to 2d array and convert it to a dataframe
m,n,r = X.shape
out_arr = np.column_stack((np.repeat(np.arange(m),n),X.reshape(m*n,-1)))
out_df = pd.DataFrame(out_arr)

In [18]:
# rename columns and drop duplicates
out_df.columns= ['index', 'I', 'II', 'III', 'aVF', 'aVR', 'aVL', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
out_df['index'] = out_df['index'].astype('int32')
out_df = out_df.drop_duplicates()
out_df

,index,I,II,III,aVF,aVR,aVL,V1,V2,V3,V4,V5,V6
0,0,-0.005,0.135,0.140,-0.065,-0.073,0.137,-0.125,-0.090,-0.110,-0.210,-0.145,-0.080
4,0,-0.005,0.135,0.140,-0.065,-0.073,0.137,-0.125,-0.090,-0.110,-0.211,-0.146,-0.080
6,0,-0.005,0.131,0.136,-0.063,-0.070,0.133,-0.125,-0.082,-0.102,-0.190,-0.129,-0.072
7,0,-0.005,0.130,0.135,-0.063,-0.070,0.132,-0.122,-0.077,-0.094,-0.172,-0.116,-0.067
8,0,-0.005,0.128,0.133,-0.062,-0.069,0.130,-0.119,-0.071,-0.084,-0.157,-0.102,-0.061
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499595,6427,0.010,0.170,0.160,-0.090,-0.075,0.165,0.155,0.365,0.230,0.030,-0.065,-0.060
4499596,6427,0.014,0.174,0.160,-0.094,-0.073,0.167,0.155,0.368,0.245,0.029,-0.057,-0.056
4499597,6427,0.016,0.176,0.160,-0.096,-0.073,0.167,0.155,0.383,0.261,0.040,-0.052,-0.055
4499598,6427,0.014,0.174,0.160,-0.094,-0.073,0.167,0.155,0.406,0.282,0.059,-0.046,-0.053


In [19]:
# read in csv file
df = pd.read_csv('training_13_features.csv')
label_df = df.copy()
label_df['unique_id'] = np.arange(label_df.shape[0])
label_df

,ritmi,age,sex,height,weight,nurse,site,device,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold,unique_id
0,2,54.0,0,166.796356,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6,0
1,1,54.0,0,166.796356,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6,1
2,0,55.0,0,166.796356,69.841845,1.0,2.0,1,1.0,1.0,0,1,0.0,10,2
3,2,29.0,1,164.000000,56.000000,7.0,1.0,10,0.0,0.0,0,1,0.0,1,3
4,2,57.0,0,166.796356,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6423,2,81.0,0,178.000000,70.000000,11.0,1.0,4,1.0,0.0,0,1,0.0,4,6423
6424,2,88.0,0,152.000000,45.000000,11.0,1.0,4,0.0,0.0,0,1,0.0,10,6424
6425,0,83.0,1,166.796356,69.841845,1.0,2.0,1,1.0,1.0,0,1,0.0,5,6425
6426,1,75.0,1,177.000000,80.000000,0.0,34.0,2,0.0,2.0,0,1,0.0,7,6426


In [20]:
# merge out_df and label_df 
merged_df = pd.merge(out_df, label_df, how='inner', left_on='index', right_on='unique_id')
merged_df = merged_df.drop(columns=['index', 'unique_id'])
merged_df

,I,II,III,aVF,aVR,aVL,V1,V2,V3,V4,...,weight,nurse,site,device,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold
0,-0.005,0.135,0.140,-0.065,-0.073,0.137,-0.125,-0.090,-0.110,-0.210,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
1,-0.005,0.135,0.140,-0.065,-0.073,0.137,-0.125,-0.090,-0.110,-0.211,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
2,-0.005,0.131,0.136,-0.063,-0.070,0.133,-0.125,-0.082,-0.102,-0.190,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
3,-0.005,0.130,0.135,-0.063,-0.070,0.132,-0.122,-0.077,-0.094,-0.172,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
4,-0.005,0.128,0.133,-0.062,-0.069,0.130,-0.119,-0.071,-0.084,-0.157,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361838,0.010,0.170,0.160,-0.090,-0.075,0.165,0.155,0.365,0.230,0.030,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8
4361839,0.014,0.174,0.160,-0.094,-0.073,0.167,0.155,0.368,0.245,0.029,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8
4361840,0.016,0.176,0.160,-0.096,-0.073,0.167,0.155,0.383,0.261,0.040,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8
4361841,0.014,0.174,0.160,-0.094,-0.073,0.167,0.155,0.406,0.282,0.059,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8


In [21]:
# dropna and reset index
new_mdf = merged_df.dropna()
new_mdf = new_mdf.reset_index(drop=True)
new_mdf

,I,II,III,aVF,aVR,aVL,V1,V2,V3,V4,...,weight,nurse,site,device,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold
0,-0.005,0.135,0.140,-0.065,-0.073,0.137,-0.125,-0.090,-0.110,-0.210,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
1,-0.005,0.135,0.140,-0.065,-0.073,0.137,-0.125,-0.090,-0.110,-0.211,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
2,-0.005,0.131,0.136,-0.063,-0.070,0.133,-0.125,-0.082,-0.102,-0.190,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
3,-0.005,0.130,0.135,-0.063,-0.070,0.132,-0.122,-0.077,-0.094,-0.172,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
4,-0.005,0.128,0.133,-0.062,-0.069,0.130,-0.119,-0.071,-0.084,-0.157,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4319171,0.010,0.170,0.160,-0.090,-0.075,0.165,0.155,0.365,0.230,0.030,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8
4319172,0.014,0.174,0.160,-0.094,-0.073,0.167,0.155,0.368,0.245,0.029,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8
4319173,0.016,0.176,0.160,-0.096,-0.073,0.167,0.155,0.383,0.261,0.040,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8
4319174,0.014,0.174,0.160,-0.094,-0.073,0.167,0.155,0.406,0.282,0.059,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8


In [22]:
# write to csv file
new_mdf.to_csv('training_25_features.csv', index=False)